In [2]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
from sklearn.metrics.pairwise import cosine_similarity, nan_euclidean_distances
from sklearn.preprocessing import LabelEncoder, normalize
import dask.array as da
import logging
from luigi import Task
from luigi.parameter import BoolParameter, IntParameter
from luigi.task import ExternalTask
from luigi.target import Target
import luigi
from csci_utils.luigi.dask.target import CSVTarget
from csci_utils.luigi.dask.target import ParquetTarget
from csci_utils.luigi.task import Requirement
from csci_utils.luigi.task import Requires
from csci_utils.luigi.task import TargetOutput
from luigi.contrib.s3 import S3Target
import pandas as pd
import pandas as pd
import numpy as np
import pathlib
from dask import dataframe as dd
from sklearn.metrics.pairwise import cosine_similarity, nan_euclidean_distances
from sklearn.preprocessing import LabelEncoder, normalize
import dask.array as da
import glob
import matplotlib
from functools import partial

DEBUG:matplotlib:(private) matplotlib data path: C:\Users\wmj\.virtualenvs\radio-star-X64HrzkT\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:matplotlib data path: C:\Users\wmj\.virtualenvs\radio-star-X64HrzkT\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\wmj\.matplotlib
DEBUG:matplotlib:matplotlib version 3.3.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', '_virtualenv', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 

In [3]:
ddf = dd.read_parquet('../data/processed/ProcessChexpertDfToParquet/part.0.parquet')

In [4]:
df = pd.read_parquet('../data/processed/ProcessChexpertDfToParquet/part.0.parquet')

In [5]:
class ChexpertDataframe(ExternalTask):

    s3_path = "s3://radio-star-csci-e-29/unzipped/"

    output = TargetOutput(
        file_pattern="", ext="train.csv", target_class=S3Target, path=s3_path
    )

class ProcessChexpertDfToParquet(Task):
    requires = Requires()
    chexpertdf = Requirement(ChexpertDataframe)

    output = TargetOutput(
        target_class=ParquetTarget,
        path="../data/processed/",
        ext="",
        flag=False,
        storage_options=dict(requester_pays=True),
    )

    def run(self):
        pathCSV = self.input()["chexpertdf"].path
        ddf = dd.read_csv(pathCSV)

        self.output().write_dask(ddf, compression="gzip")

class NormalizeDF(Task):
    """The Dataframe is best normalized before similarity calculations are
    run on it."""

    requires = Requires()
    proc_chexpertdf = Requirement(ProcessChexpertDfToParquet)

    output = TargetOutput(
        target_class=ParquetTarget,
        path="../data/processed/",
        ext="",
        flag=False,
        storage_options=dict(requester_pays=True),
    )

ddf = NormalizeDF().output().read_dask(npartitions = 2)

df = ddf.compute()



INFO:root:self.target_kwargs['path'] is ../data/processed/NormalizeDF
INFO:root:BaseDaskTarget path is ../data/processed/NormalizeDF/


unadjusted path is:  ../data/processed/NormalizeDF
read_parquet_path is:  ../data/processed/NormalizeDF\*.parquet


In [6]:
ddf.repartition(npartitions = 2)

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
npartitions=2,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
111706,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223413,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
ddf = NormalizeDF().output().read_dask()
ddf_raw = ProcessChexpertDfToParquet().output().read_dask()

INFO:root:self.target_kwargs['path'] is ../data/processed/NormalizeDF
INFO:root:BaseDaskTarget path is ../data/processed/NormalizeDF/
INFO:root:self.target_kwargs['path'] is ../data/processed/ProcessChexpertDfToParquet
INFO:root:BaseDaskTarget path is ../data/processed/ProcessChexpertDfToParquet/


unadjusted path is:  ../data/processed/NormalizeDF
read_parquet_path is:  ../data/processed/NormalizeDF\*.parquet
unadjusted path is:  ../data/processed/ProcessChexpertDfToParquet
read_parquet_path is:  ../data/processed/ProcessChexpertDfToParquet\*.parquet


In [8]:
object_cols = ddf.dtypes[(ddf.dtypes == object)].values
comparator_index = 78414
row_comparator = ddf.loc[comparator_index]

In [9]:
ddf_raw.dtypes

Path                           object
Sex                            object
Age                             int64
Frontal/Lateral                object
AP/PA                          object
No Finding                    float64
Enlarged Cardiomediastinum    float64
Cardiomegaly                  float64
Lung Opacity                  float64
Lung Lesion                   float64
Edema                         float64
Consolidation                 float64
Pneumonia                     float64
Atelectasis                   float64
Pneumothorax                  float64
Pleural Effusion              float64
Pleural Other                 float64
Fracture                      float64
Support Devices               float64
dtype: object

In [10]:
(ddf.dtypes == object)

Sex                           False
Age                           False
Frontal/Lateral               False
AP/PA                         False
No Finding                    False
Enlarged Cardiomediastinum    False
Cardiomegaly                  False
Lung Opacity                  False
Lung Lesion                   False
Edema                         False
Consolidation                 False
Pneumonia                     False
Atelectasis                   False
Pneumothorax                  False
Pleural Effusion              False
Pleural Other                 False
Fracture                      False
Support Devices               False
dtype: bool

In [11]:
object_cols = ddf_raw.dtypes[(ddf_raw.dtypes == object).values].index

In [12]:
object_cols

Index(['Path', 'Sex', 'Frontal/Lateral', 'AP/PA'], dtype='object')

In [14]:
most_similar_idx = (ddf.values == row_comparator.values).sum(axis=1).astype('int').compute().argsort()[::-1]

In [15]:
ddf_filled = ddf.fillna(-1)

In [16]:
ddf_filled.astype('int')

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
npartitions=1,,,,,,,,,,,,,,,,,,
0,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32
223413,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [17]:
idx_partition_to_view = most_similar_idx[:int(most_similar_idx.size/5)]

In [18]:
df = ddf.loc[idx_partition_to_view].compute()

In [19]:
closest_in_space = nan_euclidean_distances(df.fillna(0), df.loc[comparator_index].fillna(0).values.reshape(1,-1))

In [20]:
pd.DataFrame(data = closest_in_space, index = df.index)

,0
78414,0.000000
122415,1.414214
163825,1.133333
174937,1.443418
78408,1.414214
...,...
196645,3.038843
113073,1.988982
196642,1.926937
204572,2.057807


In [21]:
closest_in_space.argsort(axis=0)

array([[    0],
       [   43],
       [   45],
       ...,
       [16736],
       [14425],
       [40938]], dtype=int64)

In [22]:
df['dist'] = closest_in_space

In [23]:
df = df.sort_values('dist')

In [24]:
row_comparator_raw = ddf_raw.loc[comparator_index].compute()

In [25]:
row_comparator_raw

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
78414,CheXpert-v1.0-small/train/patient18824/study22...,Female,40,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN


In [26]:
discrete_cols = list(df.columns[2:-1])
discrete_cols

['Frontal/Lateral',
 'AP/PA',
 'No Finding',
 'Enlarged Cardiomediastinum',
 'Cardiomegaly',
 'Lung Opacity',
 'Lung Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural Effusion',
 'Pleural Other',
 'Fracture',
 'Support Devices']

In [27]:
row_comparator = row_comparator

In [28]:
row_comparator['Pneumonia']

Dask Series Structure:
npartitions=1
78414    float64
78414        ...
Name: Pneumonia, dtype: float64
Dask Name: getitem, 3 tasks

In [29]:
seek = - 1

In [30]:
df['Pneumonia'] == seek

78414     False
121323    False
34493     False
134189    False
78415     False
          ...  
48915      True
197343     True
159122     True
107974     True
113518     True
Name: Pneumonia, Length: 44682, dtype: bool

In [31]:
close_idx = pd.DataFrame(data = closest_in_space, index = df.index)

In [32]:
closestidx = close_idx[df['Pneumonia'] == seek].idxmin()

In [33]:
df.loc[closestidx]

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,dist
66619,-1.0,0.666667,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.570007


In [34]:
import numpy as np

def find_close_row(df, close_idx, column, value):
    closestidx = close_idx[df[column] == value] \
        .drop(row_comparator.index.values[0], errors = 'ignore') \
        .idxmin()
    row = df.loc[closestidx]
    return row

def return_df_close_rows(df, row_comparator, close_idx):
    encoded_dict = {1: "POSITIVE ", 0: "UNCERTAIN ", -1: "NEGATIVE ",
                    np.NaN: 'unmentioned'}

    df_case_control = pd.DataFrame(columns=row_comparator.columns)
    cols_hierarchy = []
    for column in row_comparator:
        if row_comparator[column].values in (-1, 0, 1):

            for value in (-1, 1):

                try:
                    row = find_close_row(df, close_idx, column, value)

                    df_case_control = pd.concat((df_case_control, row))

                except:
                    pass
            cols_hierarchy.append(column)

    mi = pd.MultiIndex.from_frame(df_case_control[cols_hierarchy])

    multi_index_images_df = pd.Series(df_case_control.index, index=mi, name = 'Id')

    return df_case_control, multi_index_images_df

In [35]:
df

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,dist
78414,-1.0,-0.111111,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,0.000000
121323,-1.0,0.444444,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,0.555556
34493,-1.0,-0.111111,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.000000
134189,-1.0,-0.133333,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000247
78415,-1.0,-0.133333,-1.0,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.000247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48915,-1.0,0.333333,-1.0,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,NaN,1.0,3.632841
197343,-1.0,0.488889,-1.0,-1.0,NaN,NaN,NaN,-1.0,1.0,1.0,NaN,-1.0,-1.0,1.0,1.0,NaN,NaN,1.0,3.655133
159122,-1.0,0.800000,-1.0,-1.0,NaN,-1.0,NaN,1.0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,NaN,NaN,NaN,3.718887
107974,-1.0,0.844444,-1.0,-1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,-1.0,-1.0,-1.0,1.0,1.0,NaN,NaN,3.730025


In [37]:
close_idx = pd.DataFrame(data = closest_in_space, index = df.index)

In [41]:
row_comparator

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
npartitions=1,,,,,,,,,,,,,,,,,,
78414,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
78414,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [36]:
find_close_row(df, close_idx, 'Pneumonia', 1)

ValueError: Arrays chunk sizes are unknown: (nan,)

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

In [ ]:
FindSimilar().run()

In [43]:
from sklearn.neighbors import NearestNeighbors

In [46]:
samples = df.fillna(-1)

In [64]:
comparator = df.loc[comparator_index]

In [44]:
neigh = NearestNeighbors(radius=1.6)

In [47]:
neigh.fit(samples)

NearestNeighbors(radius=1.6)

In [65]:
rng = neigh.kneighbors([samples.iloc[3]], n_neighbors = 2)

In [66]:
print(rng)

(array([[0.        , 0.02223455]]), array([[ 3, 14]], dtype=int64))


In [74]:
samples

,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,dist
78414,-1.0,-0.111111,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.000000
121323,-1.0,0.444444,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.555556
34493,-1.0,-0.111111,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,-1.0,1.000000
134189,-1.0,-0.133333,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000247
78415,-1.0,-0.133333,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.000247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48915,-1.0,0.333333,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,3.632841
197343,-1.0,0.488889,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,3.655133
159122,-1.0,0.800000,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,3.718887
107974,-1.0,0.844444,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,3.730025
